In [ ]:
#this creates a csv file 'universityList.csv' containing name, key and link for all university.
# Note to self
# ------- Do not Execute this cell -----------

getUniversityList()

In [71]:
import requests
import sys
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import urllib.request
from requests.exceptions import HTTPError
import pandas as pd
import numpy as np
import time

In [58]:
def open_url_2_soup(url):
    """
    open webpage and convert to beautiful soup object
    returns: page_soup(soup object)
    params: url(str) - url of the page
    
    """
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url,headers = headers,verify = False, allow_redirects = False)
    if(page.status_code == 200):
        page_soup = soup(page.content,'lxml')
        return page_soup
    else: 
        raise HTTPError()

In [3]:
def getUniversityLookup(url):
    """
    get the list of university and it's basic info from ONE page
    returns : univ_df - dataframe containing name,key and url of universities one page at a time
    params: url - url of the page
    
    """
    univ_name = []
    univ_link = []
    univ_soup = open_url_2_soup(url)
    univ_detail_tag = univ_soup.findAll("div",{"class":"col-sm-9 col-xs-12"})
    for tag in univ_detail_tag:
        a = tag.find("a",href=True)
        univ_name.append(a.text)
        univ_link.append("https://yocket.in"+a['href'])
    #data cleaning 
    univ_df = pd.DataFrame({"name":np.asarray(univ_name),"href": np.asarray(univ_link)})
    print(univ_df['name'].str.split('(',n = 1, expand = True))
    univ_df[['name','keys']] = univ_df['name'].str.split('(',n = 1, expand = True)
    univ_df['keys'] = univ_df['keys'].apply(lambda x: x[:-1])
    return univ_df

In [4]:
def getUniversityList():
    """
    creates a csv file of all university detail
    returns: UniversityList_df - dataframe containg the basic details of top 500 engineering schools
    params: None
    
    """
    UniversityList_df = pd.DataFrame()
    count = 1
    while(count <= 16):
        url = "https://yocket.in/universities?page=" + str(count)
        temp_df = getUniversityLookup(url)
#         print(temp_df)
        UniversityList_df = UniversityList_df.append(temp_df,ignore_index=True)
#         print(UniversityList_df)
        time.sleep(5) 
        count += 1
    UniversityList_df.to_csv('universityList.csv', encoding='utf-8')
    return UniversityList_df


In [5]:

def getUniversityDetail(url):
    """
    get the details of a single university
    params: url(string) - url of the site
    returns : univ_detail(dict) - dictionary containing details of a single university
    
    """
    univ_detail = {}
    page_soup = open_url_2_soup(url)
    univ_tag = page_soup.findAll("div",{"class":"col-sm-3 col-xs-6"})
    univ_detail['name'] = page_soup.findAll("h1")[0].text
    tag_no = 1;
    for tag in univ_tag:
        if(tag_no in [1,2,3,4,6,8,9]):
            name,data = extractUnivDetails(tag,tag_no)
            univ_detail[name] = data
        tag_no+= 1
    try:
        univ_detail['admitList'] = univ_detail['student_link']+"2"
        univ_detail['rejectList'] = univ_detail['student_link']+"3"
        univ_detail['appliedList'] = univ_detail['student_link']+"1"
        del univ_detail['student_link']
    except:
        univ_detail['admitList'] = None
        univ_detail['rejectList'] = None
        univ_detail['appliedList'] = None
        del univ_detail['student_link']
    return(univ_detail)
getUniversityDetail("https://yocket.in/university-reviews/california-state-university-chico-3145/engineering")

/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


NameError: name 'extractUnivDetails' is not defined

In [6]:
def extractUnivDetails(tag,tag_no):
    """
    returns : univ_dict(dictionary) - dictionary containing university(single) detail
    params: tag(soup element) - the div containing the element
            tag_no(int) - number of div
    """
    univ_dict = {}
    if(tag_no in [1,2,8,9]):
        return(tag.h3.small.text,(tag.br.previous_sibling)[1:])
    elif(tag_no ==3):
        return (tag.h3.findAll("small")[1].text,tag.h3.find("small").previous_sibling)
    elif(tag_no ==4):
        return (tag.h3.findAll("small")[1].text,(tag.h3.find("small").previous_sibling)[2:])
    elif(tag_no == 6):
        try:
            a = tag.findAll("a",href=True)[0]
        except:
            return("student_link",None)
        else:
            return("student_link",("https://yocket.in"+a['href'])[:-1])


In [7]:
def univDetail_to_csv():
    """
    generate csv file containing university details
    params : None
    returns : None
    """
    univ_list = pd.read_csv('universityList.csv')
    universityDetail = pd.DataFrame()
    counter = 51
    while(counter <= 100):
        link = univ_list.iloc[counter]['href']
        dict_temp = getUniversityDetail(link)
        universityDetail = universityDetail.append(dict_temp,ignore_index = True)
        print(counter)
        counter += 1
        time.sleep(10)
#     print(universityDetail)
    universityDetail.to_csv('universityDetailList.csv', encoding='utf-8',header = False,mode = 'a')

In [ ]:
#create the cab file of university list with all it's parameter
#  Note to self
# -----------DO NOT EXECUTE THIS CELL --------------
univDetail_to_csv()   

In [8]:
def getStdList(page_soup,admit_status):
    """
    calculates the details of user for a single page
    params : admit_status(string) - admit/reject string
             page_soup(soup object) - soup of each page of student list of a university
    returns: dataframe containing user detail from a single page
    """
    name_tag = page_soup.findAll("input" , {"id" : "users-view-search-universities"})[0]
    name = name_tag['value']
    grading = page_soup.findAll("div",{"class" : "col-sm-6"})[1:]
#     print(grading[0])
    student_df = pd.DataFrame()
    for tag in grading:
#         print(tag)
        std_data = {}
        std_data['college'] = name
        std_data['admit/reject'] = admit_status
        a = tag.findAll("a",href = True)[0]
#         print(a)
        std_data['Name'] = a.text #Tag for name of student
#         std_data['studentLink'] - a['href']
        std_data['CourseName'] = tag.find("small").text #Tag for course
        student_data_tag = tag.findAll("div",{"class":"col-sm-3 col-xs-6"})
        for data_tag in student_data_tag:
            heading = data_tag.find("strong").text
            data = data_tag.findNext("br").next_sibling
            if (heading == "GRE"):
                std_data['GRE'] = data.string

            elif(heading == "TOEFL" or heading == "IELTS" or heading == "ENG TEST"):
                std_data['ENG_TEST'] = data.string

            elif(heading == "UNDERGRAD"):
                std_data['CGPA']= data.string

            elif(heading == "WORK EX"):
                std_data['WorkExp'] = data.string
#         print(std_data)
        student_df = student_df.append(std_data, ignore_index=True)
#     print(student_df)
    return student_df


In [59]:
def getStudentDetailList(url,admit):
    """
    gets the entire student list for a single university
    params: url(string) - student list url of the university
            admit(string) - admit/reject status
    return: student_detail(dataframe) - containg all the student list for a single university
    """
    page_counter = 1
    student_detail = pd.DataFrame()
    url = url + "?page=1"
    while(True):
        try:
            page_soup = open_url_2_soup(url)
        except HTTPError:
            print(url)
            print("exception raised")
            page_counter = 1
            break
            
        else:
            print("in else")
            std_detail_page = getStdList(page_soup,admit)
            student_detail = pd.concat([student_detail,std_detail_page])
#             print(student_detail)
            page_counter += 1
            url = url[:-1] + str(page_counter)
    return student_detail

In [64]:
def generateStudentList():
    """
    generate applied and rejected student detail list for all universities
    params: None
    returns: None
    """
    student_detail_df = pd.read_csv("universityDetailList.csv", header=1)
    header = True
    for index,row in student_detail_df.iterrows():
        print(row['admitList'])
        student_admit = getStudentDetailList(row['admitList'],'admit')
        student_admit.to_csv('studentList.csv', mode = 'a', encoding='utf-8',header = header)
        header = False
        print(row['rejectList'])
        student_reject = getStudentDetailList(row['rejectList'],'rejected')
        student_reject.to_csv('studentList.csv', mode='a', encoding = 'utf-8', header = header)
        break
    
        
               

In [80]:

generateStudentList()



https://yocket.in/applications-admits-rejects/northeastern-university/2


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


in else
           CGPA                                         CourseName ENG_TEST  \
0    
9.3 CGPA            \nNortheastern University \nSpring 2019     
6.5    
1        
67 %   \nNortheastern University Information Systems\...     
96    
2        
65 %   \nNortheastern University Information Systems\...     
96    
3   
7.85 CGPA   \nNortheastern University Engineering Manageme...    
100    
4   
7.02 CGPA   \nNortheastern University Biotechnology\nSprin...     
99    
5   
7.75 CGPA   \nNortheastern University Digital Media\nSprin...    
7.5    
6        
80 %   \nNortheastern University Pharmaceutical Scien...     
NA    
7     
69.74 %   \nNortheastern University Computer Science\nSp...      
7    
8   
8.34 CGPA   \nNortheastern University Computer Science\nFa...      
8    
9   
5.21 CGPA     \nNortheastern University Analytics\nFall 2018      
NA    
10    
74.65 %   \nNortheastern University Project Management\n...      
8    
11  
6.43 CGPA   \nNortheastern University E

/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://yocket.in/applications-admits-rejects/northeastern-university/2?page=2
exception raised
https://yocket.in/applications-admits-rejects/northeastern-university/3


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


in else
           CGPA                                         CourseName ENG_TEST  \
0        
74 %              \nNortheastern University \nFall 2018      
93    
1   
7.04 CGPA   \nNortheastern University Industrial Engineeri...    
6.5    
2   
6.75 CGPA   \nNortheastern University Mechanical Engineeri...     
98    
3      
7 CGPA   \nNortheastern University Industrial Engineeri...      
7    
4      
7 CGPA   \nNortheastern University Data Science\nSpring...     
98    
5        
77 %   \nNortheastern University Computer Science\nFa...     
92    
6   
6.36 CGPA   \nNortheastern University Electrical & Compute...    
108    
7        
61 %   \nNortheastern University Industrial Engineeri...    
108    
8        
64 %   \nNortheastern University Electrical & Compute...    
102    
9   
6.13 CGPA   \nNortheastern University Industrial Engineeri...      
6    
10  
7.23 CGPA   \nNortheastern University Electrical & Compute...     
88    
11       
70 %   \nNortheastern University C

/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://yocket.in/applications-admits-rejects/northeastern-university/3?page=2
exception raised


In [83]:
URL = "https://yocket.in/applications-admits-rejects/northeastern-university/3?page=2"
headers=dict(referer=URL)
response = requests.get(URL, headers = headers)
response.content

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" />\n<script src="/cdn-cgi/apps/head/Mu6wVX4OcZ8KDPC4TGyjOacEUjg.js"></script><link href="/img/yocket.ico" type="image/x-icon" rel="icon" /><link href="/img/yocket.ico" type="image/x-icon" rel="shortcut icon" />\n<link rel="dns-prefetch" href="//static.yocket.in" />\n<link rel="dns-prefetch" href="//yocket.in" />\n<link rel="dns-prefetch" href="//maxcdn.bootstrapcdn.com" />\n<link rel="dns-prefetch" href="//fonts.gstatic.com" />\n<link rel="dns-prefetch" href="//fonts.googleapis.com" />\n<meta name="theme-color" content="#EF6C00" /><link rel="icon" sizes="192x192" href="https://static.yocket.in/images/yocket_logo.png" />\n<link rel="manifest" href="/manifest.json" />\n<noscript class=\'css-links-wrapper\'>\n\t<!--<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesome.min.css" class="js-fa-async-stylesheets" media="mean

In [79]:

import urllib.request
from bs4 import BeautifulSoup
import requests
from sklearn import tree
import pickle

temp_arr = []

#Function takes a link. Returns an array with profile details.
def crawl(url, pg):
    source_code = requests.get(url + "?page="+str(pg)).text

    soup = BeautifulSoup(source_code, "html.parser")
    repeat = True

    while(repeat):
        oldlen = len(temp_arr)
        for link in soup.find_all('div', {'class': 'col-sm-3 col-xs-6'}):
            if(link.br != None):
                temp_arr.append(str(link.br.text).lstrip())

        newlen = len(temp_arr)
        if(oldlen == newlen):
            repeat = False
        pg = pg + 1
        source_code = requests.get(url + "?page=" + str(pg))
#         soup = BeautifulSoup(source_code, "html.parser")
    return temp_arr

#Returns lists of GRE, TOEFL, % and Work Ex in a list
def lister(arr):
    tuple_arr = []
    shemparr = temp_arr

    for i in range(0, len(shemparr)):
        if (shemparr[i] == "NA " or shemparr[i] == "N.A. "):
            shemparr[i] = '0'
        shemparr[i] = float(shemparr[i].split(" ", 1)[0])

    for i in range(2, len(shemparr), 4):
        if (shemparr[i] < 4):
            shemparr[i] = shemparr[i] * 21.25
        elif (shemparr[i] < 10):
            shemparr[i] = shemparr[i] * 8.5

    # l = len(temp_arr)
    while (len(shemparr) > 0):
        o = list(shemparr[0:4])
        tuple_arr.append(o)
        shemparr = shemparr[4:]

    return tuple_arr


url = input("Enter the admit URL : ")
url = url.rstrip()

crawl(url, 1)



Enter the admit URL :  https://yocket.in/applications-admits-rejects/44-texas-a-and-m-university-college-station/2


['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']